# Part 4: Almasru - exercises
### Goals
* Learn to use Almasru library

### Short description
Almasru is a python library used to fetch easily records from SRU server. The system contains a lot of tools to find related records. The system make a backup of all requests. Unless you delete the history it will keep the results in memory.

The deduplication system is however not maintened anymore. Recommendation is to use the dedupmarcxml library: https://dedupmarcxml.readthedocs.io

### Documentation
* https://almasru.readthedocs.io
* https://github.com/Swiss-Library-Service-Platform/almasru

In [2]:
# Import libraries
from almasru.client import SruClient, SruRecord, IzSruRecord, SruRequest
from almasru.utils import check_removable_records, analyse_records
# from almasru import dedup
# from almasru.briefrecord import BriefRecFactory, BriefRec
from almasru import config_log

# Config logs
config_log()

In [3]:
# Fetch one record in NZ
SruClient.set_base_url('https://swisscovery.ch/view/sru/41SLSP_NETWORK')
record = SruRecord('991042095849705501')
print(record)

2025-09-02 01:26:14,248 - INFO - Records 1 - 1 / 1, "alma.mms_id=991042095849705501": 1
<record xmlns="http://www.loc.gov/MARC21/slim">
  <leader>00698naa a22001812c 4500</leader>
  <controlfield tag="001">991042095849705501</controlfield>
  <controlfield tag="005">20241004194331.0</controlfield>
  <controlfield tag="008">991211s1979    xxk           00| | eng d</controlfield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(swissbib)032274408-41slsp_network</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">032274408</subfield>
    <subfield code="9">ExL</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(IDSBB)001963059DSV01</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="040">
    <subfield code="a">SzZuIDS BS/BE A164</subfield>
    <subfield code="b">ger</subfield>
    <subfield code="e">kids</subfield>
    <subfield code="d">CH-ZuSLS</subfield>
  </datafield>
  <datafield

In [5]:
parent_rec = record.get_parent_rec()
parent_rec

{'MMS_ID': '991042095849705501',
 'related_records_found': True,
 'number_of_rel_recs': 1,
 'related_records': {SruRecord('991017428369705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK')},
 'fields_related_records': [{'parent_MMS_ID': '991017428369705501',
   'field': '773$w',
   'content': '(IDSBB)000010912DSV01'}]}

### Some tasks with Almasru
* Find documents with empty isbn and python in the title
* Get all IZ records of books with ISBN "9783960091691" 

In [35]:
# Find documents with isbn not empty and python in the title
r = SruRequest(query='alma.isbn all "" and title="python"')

2025-09-02 01:44:18,604 - INFO - SRU data fetched: https://swisscovery.ch/view/sru/41SLSP_NETWORK?query=alma.isbn+all+%22%22+and+title%3D%22python%22&version=1.2&operation=searchRetrieve&startRecord=1&maximumRecords=10
2025-09-02 01:44:18,612 - WARNING - SruRequest('alma.isbn all "" and title="python"', limit=10, base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'): number of available results exceed the limit provided
2025-09-02 01:44:18,617 - INFO - Records 1 - 10 / 686, "alma.isbn all "" and title="python"": 10


In [26]:
r

SruRequest('(alma.isbn all "2" or alma.isbn all "1") and title="python"', limit=10, base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK')

In [36]:
# Get all IZ records of books with ISBN "9783960091691" 
r = SruRequest(query='alma.isbn all 9783960091691')
r.records

2025-09-02 01:46:22,515 - INFO - SRU data fetched: https://swisscovery.ch/view/sru/41SLSP_NETWORK?query=alma.isbn+all+9783960091691&version=1.2&operation=searchRetrieve&startRecord=1&maximumRecords=10
2025-09-02 01:46:22,523 - INFO - Records 1 - 2 / 2, "alma.isbn all 9783960091691": 2


[SruRecord('991170725301705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'),
 SruRecord('991170737505705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK')]

In [47]:
iz_recs = dict()
for nzrec in r.records:
    izs = nzrec.get_iz_using_rec()
    iz_recs[nzrec.mms_id] = []
    print(izs)
    for iz in izs:
        iz_recs[nzrec.mms_id].append(nzrec.get_iz_record(f'https://swisscovery.ch/view/sru/{iz}'))
        

2025-09-02 01:55:25,275 - INFO - SruRecord('991170725301705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'): record used in IZ: 41SLSP_ZAW, 41SLSP_HPH, 41SLSP_PHZ, 41SLSP_FHO, 41SLSP_UBE
['41SLSP_ZAW', '41SLSP_HPH', '41SLSP_PHZ', '41SLSP_FHO', '41SLSP_UBE']
2025-09-02 01:55:25,279 - INFO - SruRecord('991170725301705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'): 5 records in IZ found
2025-09-02 01:55:25,282 - INFO - SruRecord('991170725301705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'): 5 records in IZ found
2025-09-02 01:55:25,284 - ERROR - IzSruRecord('99116811404305510', base_url='https://swisscovery.ch/view/sru/41SLSP_HPH'): number of records found: 0, should be only 1
2025-09-02 01:55:25,290 - INFO - SruRecord('991170725301705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'): 5 records in IZ found
2025-09-02 01:55:25,297 - ERROR - IzSruRecord('99116811404305510', base_url='https://swisscovery.ch/view/sru/41SLSP_P

In [48]:
iz_recs

{'991170725301705501': [IzSruRecord('99116811404305510', base_url='https://swisscovery.ch/view/sru/41SLSP_ZAW'),
  IzSruRecord('991001431114605520', base_url='https://swisscovery.ch/view/sru/41SLSP_HPH'),
  IzSruRecord('99116798803605521', base_url='https://swisscovery.ch/view/sru/41SLSP_PHZ'),
  IzSruRecord('99116795705405515', base_url='https://swisscovery.ch/view/sru/41SLSP_FHO'),
  IzSruRecord('99117227594205511', base_url='https://swisscovery.ch/view/sru/41SLSP_UBE')],
 '991170737505705501': [IzSruRecord('99117112680305508', base_url='https://swisscovery.ch/view/sru/41SLSP_UZB'),
  IzSruRecord('99116800806405515', base_url='https://swisscovery.ch/view/sru/41SLSP_FHO')]}